In [11]:
from __future__ import division
import numpy as np
import cupy as cp # You need to have a CUDA-enabled GPU to use this package!
import time

In [12]:
import pandas as pd

In [13]:
A=pd.read_csv("short_x_data.csv")
B=pd.read_csv("short_x_labels.csv")
C=pd.read_csv("final_test_set.csv")
D=pd.read_csv("final_test_label.csv")



In [14]:
print(A.shape,B.shape,C.shape,D.shape)

(2500, 500) (2500, 1) (2455, 500) (2455, 1)


In [15]:
short_x_data=A.values
short_x_labels=B.values
final_test_set=C.values
final_test_label=D.values

In [16]:
thr = [100, 100]   #The threshold of hidden and output neurons
lr = [.2,.2]    #The learning rate of hidden and ouput neurons
lamda=[0.000001,0.000001]   # The regularization penalty for hidden and ouput neurons
b=[5,48]    #The upper bound of wight initializations for hidden and ouput neurons
a=[0,0]    #The lower bound of wight initializations for hidden and ouput neurons
Nepoch = 40    #The maximum number of training epochs
NumOfClasses = 2    #Number of classes
Nlayers = 2    #Number of layers
NhidenNeurons =400   #Number of hidden neurons
Dropout=[0,0] 
tmax = 256 #Simulatin time
#GrayLevels=255 #Image GrayLevels
gamma=3 #The gamma parameter in the relative target firing calculation


In [17]:
#General settings
loading=False   # Set it as True if you want to load a pretrained model
LoadFrom= "weights_pretrained.npy"  # The pretrained model
saving=True   # Set it as True if you want to save the trained model
best_perf=0
Nnrn = [NhidenNeurons, NumOfClasses]   # Number of neurons at hidden and output layers
cats=[1,0]   # Reordering the categories

In [18]:
#General variables
x_train=[]  # To keep training images
x_train_labels=[]  # To keep training labels
x_test=[]  # To keep test images
x_test_labels=[]  # To keep test labels
W=[]#To hold the weights of hidden and output layers
firingTime=[] #To hold the firing times of hidden and output layers
Spikes=[] #To hold the spike trains of hidden and output layers
X=[] #To be used in converting firing times to spike trains
target = cp.zeros([NumOfClasses]) # To keep the target firing times of current image
FiringFrequency=[] # to count number of spikes each neuron emits during an epoch

In [19]:
x_train.append(short_x_data)
x_train_labels.append(short_x_labels)
x_train= cp.asarray(x_train)
x_train_labels = cp.asarray(x_train_labels)
x_test = cp.asarray(final_test_set)
x_test_labels = cp.asarray(final_test_label)

In [20]:
#Building the model
layerSize=[[x_train[0].shape[0],x_train[0].shape[1]], [NhidenNeurons,1],[NumOfClasses,1]]    
print(layerSize)
x = cp.mgrid[0:layerSize[0][0],0:layerSize[0][1]]# To be used in converting raw image into a spike image
print(x)
SpikeImage = cp.zeros((layerSize[0][0],layerSize[0][1],tmax+1)) # To keep spike image
# Initializing the network
np.random.seed(0)
for layer in range(Nlayers):
    W.append(cp.asarray((b[layer] - a[layer]) * np.random.random_sample((Nnrn[layer], layerSize[layer][0],layerSize[layer][1]))))
    firingTime.append(cp.asarray(np.zeros(Nnrn[layer])))
    Spikes.append(cp.asarray(np.zeros((layerSize[layer+1][0],layerSize[layer+1][1],tmax+1))))
    X.append(cp.asarray(np.mgrid[0:layerSize[layer+1][0],0:layerSize[layer+1][1]]))

SpikeList=[SpikeImage]+Spikes
print(len(SpikeList))

[[2500, 500], [400, 1], [2, 1]]
[[[   0    0    0 ...    0    0    0]
  [   1    1    1 ...    1    1    1]
  [   2    2    2 ...    2    2    2]
  ...
  [2497 2497 2497 ... 2497 2497 2497]
  [2498 2498 2498 ... 2498 2498 2498]
  [2499 2499 2499 ... 2499 2499 2499]]

 [[   0    1    2 ...  497  498  499]
  [   0    1    2 ...  497  498  499]
  [   0    1    2 ...  497  498  499]
  ...
  [   0    1    2 ...  497  498  499]
  [   0    1    2 ...  497  498  499]
  [   0    1    2 ...  497  498  499]]]
3


In [ ]:
# Start learning
for epoch in range(Nepoch):
    start_time = time.time()
    correct=cp.zeros(NumOfClasses)
    FiringFrequency=cp.zeros((NhidenNeurons))

    # Start an epoch
    for iteration in range(len(x_train)): 
        #converting input image into spiking image
        SpikeImage[:,:,:]=0
        SpikeImage[x[0],x[1],x_train[iteration]] = 1

        #Feedforward path
        for layer in range(Nlayers):
            Voltage=cp.cumsum(cp.tensordot(W[layer], SpikeList[layer]),1) # Computing the voltage
            Voltage[:,tmax]=thr[layer]+1 # Forcing the fake spike
            firingTime[layer] = cp.argmax(Voltage>thr[layer],axis=1).astype(float)+1   # Findign the first threshold crossing  
            firingTime[layer][firingTime[layer]>tmax]=tmax # Forcing the fake spike
                
            Spikes[layer][:,:,:]=0   
            Spikes[layer][X[layer][0],X[layer][1],firingTime[layer].reshape(Nnrn[layer],1).astype(int)]=1 #converting firing times to spikes

        
        FiringFrequency =  FiringFrequency + (firingTime[0] < tmax) # FiringFrequency is used to find dead neurons

        #Computing the relative target firing times
        winner=np.argmin(firingTime[Nlayers-1]) 
        minFiring=min(firingTime[layer])
        if minFiring == tmax:          
            target[:]=minFiring
            target[x_train_labels[iteration]]=minFiring-gamma
            target=target.astype(int)
        else:
            target[:]=firingTime[layer][:]
            toChange=(firingTime[layer]-minFiring)<gamma
            target[toChange]=min(minFiring+gamma,tmax)
            target[x_train_labels[iteration]]=minFiring
          

        #Backward path
        layer= Nlayers-1 # Output layer

        delta_o=(target-firingTime[layer])/tmax # Error in the ouput layer

        #Gradient normalization
        norm=cp.linalg.norm(delta_o)
        if (norm!=0):
            delta_o=delta_o/norm
            
        if Dropout[layer]>0:
                firingTime[layer][cp.asarray(np.random.permutation(Nnrn[layer])[:Dropout[layer]])]=tmax
        
        # Updating hidden-output weights
        hasFired_o=firingTime[layer-1]<firingTime[layer][:, cp.newaxis] # To find which hidden neurons has fired before the ouput neurons
        W[layer][:,:,0]-=(delta_o[:, cp.newaxis]*hasFired_o*lr[layer]) # Update hidden-ouput weights 
        W[layer]-=lr[layer]*lamda[layer]*W[layer] # Weight regularization

        # Backpropagating error to hidden neurons
        delta_h=(cp.multiply(delta_o[:, cp.newaxis]*hasFired_o, W[layer][:,:,0])).sum(axis=0) #Backpropagated errors from ouput layer to hidden layer
        

        layer= Nlayers-2 # Hidden layer     
        
        #Gradient normalization
        norm=cp.linalg.norm(delta_h)
        if (norm!=0):
            delta_h=delta_h/norm
        # Updating input-hidden weights
        hasFired_h=x_train[iteration]<firingTime[layer][:, cp.newaxis, cp.newaxis] # To find which input neurons has fired before the hidden neurons
        W[layer]-=lr[layer]*delta_h[:, cp.newaxis, cp.newaxis]*hasFired_h # Update input-hidden weights 
        W[layer]-=lr[layer]*lamda[layer]*W[layer] # Weight regularization

    #Evaluating on test samples
    correct=0 
    for iteration in range(len(x_test)):        
        SpikeImage[:,:,:]=0
        SpikeImage[x[0],x[1],x_test[iteration]] = 1
        for layer in range(Nlayers):
            Voltage=cp.cumsum(cp.tensordot(W[layer], SpikeList[layer]),1)
            Voltage[:,tmax]=thr[layer]+1
            firingTime[layer] = cp.argmax(Voltage>thr[layer],axis=1).astype(float)+1     
            firingTime[layer][firingTime[layer]>tmax]=tmax
            Spikes[layer][:,:,:]=0   
            Spikes[layer][X[layer][0],X[layer][1],firingTime[layer].reshape(Nnrn[layer],1).astype(int)]=1
        minFiringTime=firingTime[Nlayers-1].min()
        if minFiringTime==tmax:
            V=np.argmax(Voltage[:,tmax-3])
            if V==x_test_labels[iteration]:
              correct+=1
        else:
            if firingTime[layer][x_test_labels[iteration]]==minFiringTime:
                correct+=1           
    testPerf=correct/len(x_test)
     
     

    

    
    print('epoch= ', epoch,  'Perf_test= ',testPerf)
    print("--- %s seconds ---" % (time.time() - start_time))
      
        
    #To save the weights
    if saving:
      np.save("weights", W, allow_pickle=True)
      if testPerf>best_perf:
        np.save("weights_best", W, allow_pickle=True)
        best_perf=val

    #To find and reset dead neurons    
    ResetCheck = FiringFrequency <0.001*len(images)    
    ToReset = [i for i in range(NhidenNeurons) if ResetCheck[i]]
    for i in ToReset:
         W[0][i]=cp.asarray((b[0] - a[0]) * np.random.random_sample((layerSize[0][0],layerSize[0][1])) + a[0]) #r

epoch=  0 Perf_test=  1.0
--- 286.0313673019409 seconds ---
